<a href="https://colab.research.google.com/github/Amzo/pneumonia/blob/main/cnnModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Introduction

Is this notebook there is the option to either train or load 4 additional models ontop of our model that we have created. These models where to compare their accuracy to our models accuracy and to also experiment with ensembling to see how the accuracy improved, which can be tested in the ensembling section.

For the sake of speed, the training of the models can be skipped and the pretrained version can be fetched to save a substantial amount of time.

# Import Libraries (Required)

Import our essential Libraries


In [30]:
import os
import matplotlib.pyplot as plt
import pickle
import cv2
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications import InceptionV3, DenseNet121
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Input, Average
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications.vgg19 import VGG19

libaryIsImported = True;

# Fetch Dataset (Required)

Some shell script to check if the data already exists, if not clone it from git.



In [31]:
%%shell

if [ ! -d '/tmp/pneumoniaDataset' ]; then 
  mkdir '/tmp/pneumoniaDataset'
fi

if [ ! -d '/tmp/pneumoniaDataset/.git' ]; then 
  git clone "https://github.com/Amzo/xray_images" '/tmp/pneumoniaDataset/'
fi

In [32]:
dataIsFetched = True

# Datasetup (Required)

Load our data and generate additional augmented data due to the nature of the small data set. Since the data is already structured into train, test and validate folders, we don't need to split the data here.


In [33]:
def getData(trainDir, testDir, valDir):
        class_names = os.listdir(trainDir)
        class_types = len(os.listdir(trainDir))

        print('Number of classes for Classification: ',class_types)
        print(f'The class names are {class_names[0]} and {class_names[1]}')
        print('--> Count of Train Images <--')

        for i in class_names:
                print(i + ':' + str(len(os.listdir(trainDir + "/" +i))))
        print('--> Count of Test Images <--')

        for i in class_names:
                print(i + ':' + str(len(os.listdir(testDir + '/' +i))))

        print('--> Count of Validation Images <---')
        for i in class_names:
                print(i + ':' + str(len(os.listdir(valDir + '/' +i))))

        trainDatagen = ImageDataGenerator(rescale=1./255.0,
                rotation_range=6,
                zoom_range=0.3,
                brightness_range= [0.8, 1.3],
                horizontal_flip = True,
                vertical_flip = True
                )

        testDatagen = ImageDataGenerator(rescale=1./255.0)

        xTrainGen = trainDatagen.flow_from_directory(
                trainDir,
                target_size=(224,224),
                shuffle=True,
                batch_size=24,
                class_mode='categorical'
        )

        xTestGen = testDatagen.flow_from_directory(
                testDir,
                target_size=(224,224),
                batch_size=16,
                shuffle=True,
                class_mode='categorical'
        )

        xValGen = trainDatagen.flow_from_directory(
                valDir,
                target_size=(224,224),
                batch_size=32,
                class_mode='categorical'
        )

        return xTrainGen, xTestGen, xValGen

In [34]:
try:
  dataIsFetched
except NameError:
  print("Data has not been fetched, run the fetch data cells")
else:
  physical_devices = tf.config.experimental.list_physical_devices('GPU')
  assert len(physical_devices) > 0, "Not enough GPU hardware devices available"


  inputTrain = '/tmp/pneumoniaDataset/train'
  inputTest = '/tmp/pneumoniaDataset/test'
  inputValidate = '/tmp/pneumoniaDataset/val'

  imageSize = (224,224,3)


  xTrain, xTest, xVal = getData(inputTrain, inputTest, inputValidate)

  dataIsSetup = True


Number of classes for Classification:  2
The class names are NORMAL and PNEUMONIA
--> Count of Train Images <--
NORMAL:1305
PNEUMONIA:3851
--> Count of Test Images <--
NORMAL:234
PNEUMONIA:390
--> Count of Validation Images <---
NORMAL:44
PNEUMONIA:32
Found 5156 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 76 images belonging to 2 classes.


# Data Visualisation (Optional)

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

In [ ]:
try:
  dataIsSetup
except NameError:
  print("please execute the dataSetup section before training")
else:
  augmentedImages = [xTrain[0][0][0] for i in range(5)]

In [ ]:
plotImages(augmentedImages)

# Model Setup (Optional)

In [45]:
def myModel():
  print("Defaulting to basic CNN")
  model = Sequential()
  model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(224,224,3)))
  model.add(Conv2D(32, (3, 3), activation='relu'))
  model.add(Conv2D(16, (3, 3), activation='relu'))
  model.add(Flatten())
  model.add(Dense(2,activation='softmax'))

  model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
    loss="categorical_crossentropy",
    metrics=["accuracy"])
  
  return model

In [7]:
def modelBuild(inputShape, modelType):
  model = Sequential()
  if (modelType == "xception"):
    print("Setting up xception model")
    xception = Xception(include_top=False,
      weights= 'imagenet',
      input_shape=inputShape,
    )
    model.add(xception)

  elif (modelType == "resnet"):
    print("Setting up resnet50 model")
    resnet = ResNet50V2(include_top=False,
      weights= 'imagenet',
      input_shape=inputShape,
    )
    model.add(resnet)

  elif (modelType == "vgg19"):
    print("Setting up vgg19 model")
    vgg = VGG19(include_top=False,
      weights= 'imagenet',
      input_shape=inputShape,
    )
    model.add(vgg)
  elif (modelType == "inception"):
    print("Setting up inception model")
    inception = InceptionV3(include_top=False,
      weights= 'imagenet',
      input_shape=inputShape,
    )
    model.add(inception)
  elif (modelType == "densenet"):
    print("Setting up densenet model")
    densenet = DenseNet121(include_top=False,
      weights= 'imagenet',
      input_shape=inputShape,
    )
    model.add(densenet)

  model.add(Flatten())
  model.add(Dense(512, activation="relu"))
  model.add(Dense(2,activation="softmax"))

  model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
    loss="categorical_crossentropy",
    metrics=["accuracy"])

  return model


In [ ]:
try:
  libaryIsImported, dataIsSetup
except NameError:
  print("Libary has not been imported, or Data has not been setup, please run the libary and Data setup cells.")
else:
  xception = modelBuild(imageSize, 'xception')
  vgg19 = modelBuild(imageSize, 'vgg19')
  resnet = modelBuild(imageSize, 'resnet')
  inception = modelBuild(imageSize, 'inception')
  densenet = modelBuild(imageSize, 'densenet')
  myCNN = myModel()

  modelsSetup = True

# Train Models (Optional)

In [46]:
checkoutPath = '/tmp/Best'

modelCheckpointCallback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkoutPath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [48]:
def modelTrain(model, xTrain, xVal, batchSize):
  history = model.fit(xTrain,
    epochs=10,
    validation_data=xVal,
    verbose=1,
    batch_size=batchSize,
    callbacks=[modelCheckpointCallback]
  )

  return history, model

I could create a function to check if the models have been setup, but you can never be certain if a cell has been executed or not, so rely on global variables being true or false. This means duplication, but it's should reduce errors.

Train the Inception Model

In [ ]:
try:
  modelsSetup
except NameError:
  print("please execute the model Setup section before training")
else:
  inceptionHistoy, trainedInception = modelTrain(inception, xTrain, xVal, 32)
  trainedInception.load_weights(checkoutPath)


Train the VGG model

In [ ]:
try:
  modelsSetup
except NameError:
  print("please execute the model Setup section before training")
else:
  vggHistory, trainedVGG = modelTrain(vgg19, xTrain, xVal, 32)
  trainedVGG.load_weights(checkoutPath)


Train the resnet model

In [ ]:
try:
  modelsSetup
except NameError:
  print("please execute the model Setup section before training")
else:
  resnetHistory, trainedResnet = modelTrain(resnet, xTrain, xVal, 32)
  trainedResnet.load_weights(checkoutPath)


Train our CNN model

In [ ]:
try:
  modelsSetup
except NameError:
  print("please execute the model Setup section before training")
else:
  ourCNNHistory, trainedCNN = modelTrain(myCNN, xTrain, xVal, 32)
  trainedCNN.load_weights(checkoutPath)

In [ ]:
trainedCNN.load_weights(checkoutPath)

Train the xception model

In [ ]:
try:
  modelsSetup
except NameError:
  print("please execute the model Setup section before training")
else:
  xceptionHistory, trainedXception = modelTrain(xception, xTrain, xVal, 32)
  trainedXception.load_weights(checkoutPath)

Train the densenet model

In [ ]:
try:
  modelsSetup
except NameError:
  print("please execute the model Setup section before training")
else:
  densenetHistory, trainedDensenet = modelTrain(densenet, xTrain, xVal, 32)
  trainedDensenet.load_weights(checkoutPath)

# Save Trained Models (Optional)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
try:
  inceptionHistory.History
except (AttributeError, NameError):
  print("Have you trained the Inception model to save?")
else:
  trainedInception.save('/content/gdrive/MyDrive/Inception')

  with open('/content/gdrive/MyDrive/inceptionHistory.pickle', 'wb') as outFile:
    pickle.dump(inceptionHistoy.history, outFile)

In [ ]:
try:
  vggHistory.history
except (AttributeError, NameError):
  print("Have you trained the VGG model to save?")
else:
  trainedVGG.save('/content/gdrive/MyDrive/VGG')

  with open('/content/gdrive/MyDrive/vggHistory.pickle', 'wb') as outFile:
    pickle.dump(vggHistory.history, outFile)
  

In [ ]:
try:
  resnetHistory.History
except (AttributeError, NameError):
  print("Have you trained the resnet model to save?")
else:
  trainedResnet.save('/content/gdrive/MyDrive/Resnet')
  
  with open('/content/gdrive/MyDrive/resnetHistory.pickle', 'wb') as outFile:
    pickle.dump(resnetHistory.history, outFile)

In [28]:
try:
  ourCNNHistory.history
except (AttributeError, NameError):
  print("Have you trained myCNN model to save?")
else:
  trainedCNN.save('/content/gdrive/MyDrive/myCNN')

  with open('/content/gdrive/MyDrive/ourCNNHistory.pickle', 'wb') as outFile:
    pickle.dump(ourCNNHistory.history, outFile)

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/myCNN/assets


In [ ]:
try:
  xceptionHistory.history
except (AttributeError, NameError):
  print("Have you trained the Xception model to save?")
else:
  trainedXception.save('/content/gdrive/MyDrive/xception')

  with open('/content/gdrive/MyDrive/xceptionHistory.pickle', 'wb') as outFile:
    pickle.dump(xceptionHistory.history, outFile)

Have you trained the Xception model to save?


In [ ]:
try:
  densenetHistory.history
except (AttributeError, NameError):
  print("Have you trained the densenet model to save?")
else:
  trainedDensenet.save('/content/gdrive/MyDrive/densenet')

  with open('/content/gdrive/MyDrive/denseNetHistory.pickle', 'wb') as outFile:
    pickle.dump(densenetHistory.history, outFile)

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/densenet/assets


# Fetch Trained Models (Required)

This shell script will fetch all the models and history from my one drive so that it can be loaded. Will save an hour or two over waiting for training to finish.

Shell variables don't appear to be reusable from other cells, so perform all the shell script in one cell.

In [29]:
%%shell
function extractModel() {
  outDIR="/tmp/trainedModels/"

  # check if the directory exits, if not create it
  if [ ! -d $outDIR ]; then 
    mkdir -p $outDIR; 
  fi

  # extract the model
  echo "extracting ${1} to ${outDIR}"
  unzip -q -o -d $outDIR $1 
}

function getFile() {
  googleURL="https://docs.google.com/uc?export=download"
  echo "fetching file: ${2}"
  sleep 3
  wgetCMD="wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate"
  fileID="$1"
  outFILE="/tmp/$2"

  if [ -f $outFILE ]; then
    rm -f $outFILE
  fi
  
  wget  -q --load-cookies /tmp/cookies.txt "$googleURL&confirm=$($wgetCMD "$googleURL&id=$fileID" -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=$fileID" -O $outFILE
  rm -rf /tmp/cookies.txt
  
  if [ ${outFILE##*.} == "zip" ]; then
    extractModel $outFILE
  fi
}

# inception model
getFile "185GypHJV8sTcL3siYXinoClbXP0OEYr1" "trainedInception.zip"

# inception history
getFile "1kqAMFgH5H-_k1cKYuuatSuZiG-xtajh_" "inceptionHistory.pickle"

# vgg model
getFile "163-EY3aU6_7OP0Ds2Gq4bIu9r84f4txJ" "trainedVgg.zip"

# vgg history
getFile "1-0H4ZjHitt8FMC1ZjZun1wlXrFMjFE5N" "vggHistory.pickle"

# resnet model
getFile "1E4pHn5-4-9VXDIdlk9qFZya1TjL_u3P-" "trainedResnet.zip"

# resnet history
getFile "1-0nN6H7lMCNPD4tlS_CXD1JMEXPcFnc9" "resnetHistory.pickle"

# myCNN model
getFile "1CA9XVWkRQ_JgMQ3y3eNcMgTB3tTdDANm" "trainedCNN.zip"

# myCNN history
getFile "1-BFAv4_m3fwlYLgWCWbziSQPplKlGfvQ" "ourCNNHistory.pickle"

# xception model
getFile "1qqRKAyN5lowZ3VE8cQZWBU54eP-QA9q-" "trainedXception.zip"

# xception history
getFile "1-HHmEafOoAMlYXngAS8sjagA-Evlc3n9" "xceptionHistory.pickle"

# denseNet model
getFile "109dzTKUYszKO8SmKIh3f4bPT0nPsoN-5" "densenet.zip"

# denseNet history
getFile "1-QorYN-kWmrrrLKUTQlALFoLPCAH0eBK" "densenetHistory.pickle"

fetching file: trainedInception.zip
extracting /tmp/trainedInception.zip to /tmp/trainedModels/
fetching file: inceptionHistory.pickle
fetching file: trainedVgg.zip
extracting /tmp/trainedVgg.zip to /tmp/trainedModels/
fetching file: vggHistory.pickle
fetching file: trainedResnet.zip
extracting /tmp/trainedResnet.zip to /tmp/trainedModels/
fetching file: resnetHistory.pickle
fetching file: trainedCNN.zip
extracting /tmp/trainedCNN.zip to /tmp/trainedModels/
fetching file: ourCNNHistory.pickle
fetching file: trainedXception.zip
extracting /tmp/trainedXception.zip to /tmp/trainedModels/
fetching file: xceptionHistory.pickle
fetching file: densenet.zip
extracting /tmp/densenet.zip to /tmp/trainedModels/
fetching file: densenetHistory.pickle


In [38]:
modelsAreFetched = True

# Load Models (Required)

In [39]:
historyList = ["inceptionHistory", "vggHistory", "resnetHistory", "ourCNNHistory", "xceptionHistory", "densenetHistory"]

In [40]:
modelList = ["Inception", "Resnet", "VGG", "myCNN", "xception", "densenet"]

In [41]:
try:
  modelsAreFetched
except NameError:
  print("You need to fetch the models to load them")
else:
  for pickleFile in historyList:
    with open('/tmp/' + pickleFile + '.pickle', 'rb') as input:
      if (pickleFile == "inceptionHistory"):
        print("Loading history file: " + pickleFile)
        inceptionHistory = pickle.load(input)
      elif (pickleFile == "resnetHistory"):
        print("Loading history file: " + pickleFile)
        resnetHistory = pickle.load(input)
      elif (pickleFile == "vggHistory"):
        print("Loading history file: " + pickleFile)
        vggHistory = pickle.load(input)
      elif (pickleFile == "ourCNNHistory"):
        print("Loading history file: " + pickleFile)
        myCNNHistory = pickle.load(input)
      elif (pickleFile == "xceptionHistory"):
        print("Loading history file: " + pickleFile)
        xceptionHistory = pickle.load(input)
      elif (pickleFile == "densenetHistory"):
        print("Loading history file: " + pickleFile)
        xceptionHistory = pickle.load(input)

      historyIsFetched = True

Loading history file: inceptionHistory
Loading history file: vggHistory
Loading history file: resnetHistory
Loading history file: ourCNNHistory
Loading history file: xceptionHistory
Loading history file: densenetHistory


In [43]:
try:
  modelsAreFetched
except NameError:
  print("You have to fetch the models before you can load them")
else:
  for trainedModel in modelList:
    print("Loading model " + trainedModel)
    if (trainedModel == "Inception"):
      trainedInception = tf.keras.models.load_model('/tmp/trainedModels/' + trainedModel)
    elif (trainedModel == "Resnet"):
      trainedResnet = tf.keras.models.load_model('/tmp/trainedModels/' + trainedModel)
    elif (trainedModel == "VGG"):
      trainedVGG = tf.keras.models.load_model('/tmp/trainedModels/' + trainedModel)
    elif (trainedModel == "myCNN"):
      trainedCNN = tf.keras.models.load_model('/tmp/trainedModels/' + trainedModel)
    elif (trainedModel == "xception"):
      trainedXception = tf.keras.models.load_model('/tmp/trainedModels/' + trainedModel)
    elif (trainedModel == "densenet"):
      trainedDensenet = tf.keras.models.load_model('/tmp/trainedModels/' + trainedModel)

Loading model Inception
Loading model Resnet
Loading model VGG
Loading model myCNN


ResourceExhaustedError: ignored

# Test Models

In [22]:
def evaluateModel(model, testSet):
  ev = model.evaluate(testSet)
  print("\n%s: %.f%%" % (model.metrics_names[1], ev[1]*100))

In [ ]:
try:
  dataIsSetup
except NameError:
  print("Please run datasetup cells first")
else:
  for trainedModel in modelList:
      print("Running " + trainedModel + " against test set")
      if (trainedModel == "Inception"):
        evaluateModel(trainedInception, xTest)
      elif (trainedModel == "Resnet"):
        evaluateModel(trainedResnet, xTest)
      elif (trainedModel == "VGG"):
        evaluateModel(trainedVGG, xTest)
      elif (trainedModel == "myCNN"):
        pred = evaluateModel(trainedCNN, xTest)
      elif (trainedModel == "xception"):
        evaluateModel(trainedXception, xTest)
      elif (trainedModel == "densenet"):
        evaluateModel(trainedDensenet, xTest)

# Model History Visualisation

In [ ]:
def visualiseHistory(historyFile):
  try:
    plt.plot(historyFile.history['accuracy'], label='accuracy')
    plt.plot(historyFile.history['val_accuracy'], label = 'val_accuracy')
  except AttributeError:
    plt.plot(historyFile['accuracy'], label='accuracy')
    plt.plot(historyFile['val_accuracy'], label = 'val_accuracy')

  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.ylim([0.5, 1])
  plt.legend(loc='lower right')

  test_loss, test_acc = trainedCNN.evaluate(xTest, verbose=2)

In [ ]:
def visualiseLoss(historyFile):
  # summarize history for loss
  try:
    plt.plot(historyFile.history['loss'])
    plt.plot(historyFile.history['val_loss'])
  except AttributeError:
    plt.plot(historyFile['loss'])
    plt.plot(historyFile['val_loss'])

  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()

In [ ]:
visualiseHistory(inceptionHistory)

In [ ]:
visualiseLoss(inceptionHistory)

In [ ]:
visualiseHistory(vggHistory)

In [ ]:
visualiseLoss(vggHistory)

In [ ]:
visualiseHistory(myCNNHistory)

In [ ]:
visualiseLoss(myCNNHistory)

In [ ]:
visualiseHistory(resnetHistory)

In [ ]:
visualiseLoss(resnetHistory)

In [ ]:
visualiseHistory(xceptionHistory)

In [ ]:
visualiseLoss(xceptionHistory)

# Ensembling

In [ ]:
def getVoting():
  models = list()
  models.append(trainedInception)
  models.append(trainedVGG)
  models.append(trainedResnet)
  models.append(trainedCNN)
  models.append(trainedXception)
  models.append(trainedDensenet)

  return models

In [ ]:
combinedModel = getVoting()

In [ ]:
modelInput = Input(shape=(224, 224, 3))
modelOutputs = [model(modelInput) for model in combinedModel]
ensembleOutput = Average()(modelOutputs)

In [ ]:
ensembleModel = Model(inputs=modelInput, outputs=ensembleOutput, name='ensemble')

In [ ]:
ensembleModel.compile(optimizer='adam',
    loss="categorical_crossentropy",
    metrics=["accuracy"])

In [ ]:
print("Running ensembled model against test set")
evaluateModel(ensembleModel, xTest)

# Optional Data

Shell script can't be reused, so redeclare getFile() here, I could either fetch the optional data with the model, or redo the function in python.

In [24]:
%%shell
function getFile() {
  googleURL="https://docs.google.com/uc?export=download"
  echo "fetching file: ${2}"
  wgetCMD="wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate"
  fileID="$1"
  outFILE="/tmp/$2"

  if [ -f $outFILE ]; then
    rm -f $outFILE
  fi
  
  wget  -q --load-cookies /tmp/cookies.txt "$googleURL&confirm=$($wgetCMD "$googleURL&id=$fileID" -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=$fileID" -O $outFILE
  rm -rf /tmp/cookies.txt
}

getFile "1OMzk9acsG07iKkEn6P71evY1kvmScwFD" "optionalData.zip"

outDIR="/tmp/optionalData/"
if [ ! -d $outDIR ]; then 
  mkdir -p $outDIR; 
fi
unzip -q -o -d $outDIR "/tmp/optionalData.zip" 

fetching file: optionalData.zip


This is my sanity check to ensure that all the optional data has never been seen before.

In [ ]:
%%shell

hashFile="/tmp/hashes.txt"
optionalDataDir="/tmp/optionalData"
originalDataDir="/tmp/pneumoniaDataset"

if [ -f "${hashFile}" ]; then
  rm "${hashFile}"
fi

if [ -d "${optionalDataDir}" ] || [ -d "${originalDataDir}" ]; then
  echo "Caching all file hashes"
  for x in $(ls ${originalDataDir}/*/*/* ); do
      newMD5=$(md5sum ${x} | awk -F ' ' '{print $1}')
      echo "${newMD5}:${x}" >> ${hashFile}
  done
  echo "Caching done"

  echo "Checking optional data against cached hashes"
  for i in $(ls ${optionalDataDir}/*/*); do
    savedMD5=$(md5sum ${i} | awk -F ' ' '{print $1}')
    if $(grep $savedMD5 /tmp/hashes.txt); then
      fileName=$(grep $savedMD5 ${hashFile} | awf -F ':' '{print $2}')
      echo "Duplicate files ${i} and ${fileName}"
    fi
  done
  echo "Check Complete"
else
  echo "Has the optional data and normal data been fetched?"
fi


In [ ]:
testData = '/tmp/optionalData'
class_names = os.listdir(testData)
class_types = len(os.listdir(testData))

print('Number of classes for Classification: ',class_types)
print(f'The class names are {class_names[0]} and {class_names[1]}')
print('--> Count of Train Images <--')

for i in class_names:
  print(i + ':' + str(len(os.listdir(testData + '/' +i))))

testDatagen = ImageDataGenerator(rescale=1./255.0)

optionalTest = testDatagen.flow_from_directory(
  testData,
  target_size=(224,224),
  batch_size=16,
  shuffle=True,
  class_mode='categorical'
)

In [ ]:
evaluateModel(trainedCNN, optionalTest)

In [ ]:
evaluateModel(trainedVGG, optionalTest)

In [ ]:
evaluateModel(trainedInception, optionalTest)

In [ ]:
evaluateModel(trainedResnet, optionalTest)

In [ ]:
evaluateModel(trainedXception, optionalTest)

In [ ]:
evaluateModel(trainedDensenet, optionalTest)

In [ ]:
evaluateModel(ensembleModel, optionalTest)

# Predict Image

Function to predict image classification for reuseability.

In [ ]:
def predictImg(img):
  image = cv2.imread(img)
  image = cv2.resize(image,(224,224))
  image = np.asarray(image)
  image = np.expand_dims(image, axis=0)
        
  for model in combinedModel:
    pred = np.argmax(model.predict(image), axis=-1)
    print("Normal" if pred == 0 else "Pneumonia")


Image locations are passed in using the healthy and pneum variables and their classifcation is determined.

In [ ]:
healthy = '/tmp/pneumoniaDataset/test/NORMAL/IM-0109-0001.jpeg';
pneum = '/tmp/pneumoniaDataset/test/PNEUMONIA/person101_bacteria_483.jpeg';

print("Healthy image classification")
predictImg(healthy)
print("Pneumonia image classification")
predictImg(pneum)